# Eigenmode and EPR analysis
### Prerequisite
You need to have a working local installation of Ansys.

In [7]:
%reload_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr


from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl import TransmonCrossFL_CL
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.couplers.tunable_coupler_01 import TunableCoupler01

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround

from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.lumped.cap_3_interdigital import Cap3Interdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

#from qiskit_metal.qlibrary.terminations.launchpad_wb_simplecoupler import LaunchpadWirebondSimple
#from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
import numpy as np
from collections import OrderedDict

from qiskit_metal.qlibrary.qubits.transmon_cross_ebru_temp import TransmonCross_ebru_temp

from qiskit_metal.qlibrary.qubits.transmon_cross_fl_ebru_temp import TransmonCrossFL_ebru_temp
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl_ebru_temp import TransmonCrossFL_CL_ebru_temp

In [8]:
design = metal.designs.DesignPlanar(overwrite_enabled=True)

design.chips.main.size['size_x'] = '5mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['size_z'] = '-725um'
gui = MetalGUI(design)

Create a single transmon with one readout resonator and move it to the center of the chip previously defined.

In [9]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"


find_resonator_length(6.0, 10, 6, 4)

'5.071952814490852 mm'

In [10]:
# design.overwrite_enabled = True
options =  dict(
    connection_pads=dict(
        readout = dict(loc_W=+1, loc_H=+1),
        storage = dict(loc_W=-1, loc_H=-1, )
    ))
    
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl import TransmonCrossFL_CL

Q1 = TransmonCrossFL_CL_ebru_temp(design, 'Q1', options = dict(make_extra_rect=False, pos_x = '0.450mm', pos_y='1.410mm', 
                                                     cross_length='40um', cross_width='10um', cross_gap='15um',
                                                    # cross_gap='3um',
                                                 connection_pads = dict(
                                                     readout = dict(connector_location = '90', lead_gap='6um',
                                                     claw_length='30um', claw_width='5um', ground_spacing='2um', 
                                                                    claw_gap='5um', lead_width='10um', lead_length='60um')),#claw_length ='115um', claw_width='25um', claw_gap='7um',ground_spacing='3um')),
                                                 fl_options = dict(t_shift='3.5um' , t_offset='-6um', t_inductive_gap='4.5um', 
                                                                   t_top='17.5um', t_width='5um', t_gap='3um'),
                                                 cl_options= dict(c_offset='-40um', c_ground_gap='38um', c_width='5um', 
                                                                  c_gap='3um', translate_x='0um', c_angle='90'),
                                                 ))
    


Q2 = TransmonCrossFL_CL(design, 'Q2', options = dict(make_extra_rect=True, pos_x = '0.2mm', pos_y='1.410mm', orientation=0, 
                                                     cross_length='160um', cross_width='30um', cross_gap='30um',
                                                    
                                                 connection_pads = dict(
                                                     readout = dict(connector_location = '90', lead_gap='6um',
                                                     claw_length='140um', claw_width='10um', ground_spacing='3um', 
                                                                    claw_gap='8um', lead_width='10um', lead_length='60um')),#claw_length ='115um', claw_width='25um', claw_gap='7um',ground_spacing='3um')),
                                                 fl_options = dict(t_shift='3.5um' , t_offset='-6um', t_inductive_gap='4.5um', 
                                                                   t_top='17.5um', t_width='5um', t_gap='3um'),
                                                 cl_options= dict(c_offset='30um', c_ground_gap='38um', c_width='5um', 
                                                                  c_gap='3um', translate_x='0um', c_angle='270'),
                                                 rect_options= dict(rect_width='40um', rect_length='8um')))

# gui.rebuild()
# gui.autoscale()

In [11]:
import numpy as np
from collections import OrderedDict


anchors1f = OrderedDict()
anchors1f[0] = np.array([-1.75, -2.0])



launch_flux_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                           pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um', 
                           pos_x='-1.5mm', pos_y='0.5mm', orientation='0', lead_length='30um')
lf = LaunchpadWirebond(design, 'Launch_Flux', options = launch_flux_options)


flux_line_options = Dict(
        #lead = dict(start_straight='100um', end_straight='100um'),
        fillet='30um', 
        trace_width = '5um', trace_gap = '3um', 
        pin_inputs=Dict(
            start_pin=Dict(component='Q2', pin='flux_line'),
            end_pin=Dict(componentl=lf.name, pin='tie')), )
flux_line_Q2 = RouteAnchors(design,'Flux_Line_1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_Flux',
                                                    pin='tie')),
                                            lead=Dict(
#                                                 start_straight='175um',
                                                 #end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                                    fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                             #anchors = anchors1f
                                            ))




# anchors2fl = OrderedDict()
# anchors2fl[0] = np.array([1.78, 0.415])



launch_flux2_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.5', pos_y='0.5mm', orientation='180', lead_length='30um')
lf2 = LaunchpadWirebond(design, 'Launch_Flux2', options = launch_flux2_options)




flux_line_Q2 = RoutePathfinder(design,'Flux_Line_2', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_Flux2',
                                                    pin='tie')),
                                            lead=Dict(
#                                                 start_straight='175um',
                                                 end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors=anchors2fl
                                            ))


# gui.rebuild()

In [12]:




launch_charge_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.5mm', pos_y='1.41mm', orientation='180', lead_length='30um')

lc = LaunchpadWirebond(design, 'Launch_Charge', options = launch_charge_options)






launch_charge2_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='-1.5mm', pos_y='1.41mm', orientation='0', lead_length='30um')

lc2 = LaunchpadWirebond(design, 'Launch_Charge2', options = launch_charge2_options)


#anchors1ch = OrderedDict()
#anchors1ch[0] = np.array([1.78, 0.114])


charge_line_Q1 = RoutePathfinder(design,'Charge_Line_1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='Launch_Charge',
                                                    pin='tie')),
                                            lead=Dict(
                                                start_straight='175um',
                                                end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors1ch
                                            ))

anchors2ch = OrderedDict()
anchors2ch[0] = np.array([-1.75,2.0])
charge_line_Q2 = RoutePathfinder(design,'Charge_Line_2', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='Launch_Charge2',
                                                    pin='tie')),
                                            lead=Dict(
#                                                 start_straight='175um',
                                                 end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors2ch
                                            ))

# gui.rebuild()


04:14PM 15s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 15s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 15s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 15s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [13]:

TQ2 = CoupledLineTee(design, 'TQ2', options=dict(pos_x='-0.85mm',
                                             pos_y='2.6mm',
                                             coupling_length='410um', coupling_space='4um', orientation=0, 
                                                 open_termination=False))





#3.58

options2 = Dict(
     total_length='4.68mm',
     hfss_wire_bonds = True,
     fillet='40um',
     lead = dict(start_straight='250um'),
     pin_inputs=Dict(
         end_pin=Dict(
             component='TQ2',
             pin='second_end'),
         start_pin=Dict(
             component='Q2',
             pin='readout')),
     meander=Dict(spacing='200um',
             asymmetry='0um'))
read_q2 = RouteMeander(design, 'read_q2', options=options2)



# gui.rebuild()
# gui.autoscale()


In [14]:

TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='1.07mm',
                                             pos_y='2.6mm',
                                              coupling_length='400um', coupling_space='4um', open_termination=False, orientation='0'))

options1 = Dict(
    total_length='5.07mm',
    hfss_wire_bonds = True,
    fillet='40um',
    lead = dict(start_straight='100um', end_straight='20um'),
    pin_inputs=Dict(
        start_pin=Dict(
            component='TQ1',
            pin='second_end'),
        end_pin=Dict(
            component='Q1',
            pin='readout')))
read_q1 = RouteMeander(design, 'read_q1', options=options1)
gui.rebuild()
gui.autoscale()

04:14PM 17s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 17s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 17s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 17s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [15]:
launch_feed_left_options = dict(trace_width= '10um',pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='-1.5mm', pos_y='2.6mm', orientation='0', lead_length='30um')
l_feed_left = LaunchpadWirebond(design, 'Launch_Feed_Left', options = launch_feed_left_options)


launch_feed_right_options = dict(trace_width= '10um', pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.5mm', pos_y='2.6mm', lead_length='30um', orientation='180')
l_feed_right = LaunchpadWirebond(design, 'Launch_Feed_Right', options = launch_feed_right_options)

(optional) Captures the renderer GUI

In [16]:
%metal_heading Now analyzing the design: HFSS & EPR

(optional) Work directly with the convergence numbers

In [17]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

eig_qb = EPRanalysis(design, "hfss")

eig_qb.sim.setup.vars.Lj = '12 nH'
eig_qb.sim.setup.vars = Dict({'Lj': '12 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

04:14PM 23s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 23s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 23s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 23s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


(optional) You can re-run the analysis after varying the parameters.<br>
Not passing the parameter `components` to the `sim.run()` method, skips the rendering and tries to run the analysis on the latest design. If a design is not found, the full metal design is rendered.

In [18]:
design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



04:14PM 25s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 25s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 25s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 25s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


Verify that the Electro(magnetic) fields look realistic.

In [19]:

design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



eig_qb.sim.run(name="Xmon_only_new", components=['Q2'], open_terminations=[], box_plus_buffer = False)
eig_qb.sim.plot_convergences()

04:14PM 27s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 27s WARNING [check_lengths]: For path table, component=Charge_Line_1, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 27s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:14PM 27s WARNING [check_lengths]: For path table, component=Charge_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
INFO 04:14PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:14PM [load_ansys_project]: 	Opened Ansys App
INFO 04:14PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 04:14PM [load_ansys_project]

(optional) clear the renderer by removing the fields

In [ ]:
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'
eig_qb.run_epr()

### EPR Analysis
#### Setup
Identify the non-linear (Josephson) junctions in the model. You will need to list the junctions in the epr setup.

In this case there's only one junction, namely 'jj'. Let's see what we need to change in the default setup.

In [24]:
eig_qb.setup

{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q1_rect_jj',
   'line': 'JJ_Lj_Q1_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj'}

The name of the `Lj_variable` and `Cj_variable` match with our model. However it is missing the names of the shapes that identify the junction (`rect` and `line`). Look for those in the renderer and find the name. Then let's change the name (See below).

In [25]:
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'
eig_qb.setup

{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q1_rect_jj',
   'line': 'JJ_Lj_Q1_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj'}

We will now run epr as a single step. On screen you will observe various information in this order:
* stored energy = Electric and magnetic energy stored in the substrate and the system as a whole.
* EPR analysis results for all modes/variations.
* Spectrum analysis.
* Hamiltonian report.

In [26]:
eig_qb.run_epr()

#### equivalent individual calls
# s = self.setup
# self.epr_start()
# eig_qb.get_stored_energy()
# eig_qb.run_analysis()
# eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc)
# eig_qb.report_hamiltonian(s.swp_variable)

AssertionError: pyEPR ProjectInfo user error found 😷:
                    Seems like for junction `jj` you specified a rect that does not exist
                    in HFSS by the name: `JJ_rect_Lj_Q1_rect_jj` 

# 2. Analyze the CPW resonator by itself
### Update the design in Metal
Connect the transmon to a CPW. <br>
The other end of the CPW connects to an open to ground termination.

In [ ]:
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
RouteMeander(design, 'readout',  Dict(
        total_length='6 mm',
        hfss_wire_bonds = True,
        fillet='90 um',
        lead = dict(start_straight='100um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

### Finite Element Eigenmode Analysis

#### Setup

Create a separate analysis object, dedicated to the readout. This allows to retain the Qubit session active, in case we will later need to tweak the design and repeat the simulation. When different renderers are available you could even consider using different more appopriate ones for each simulation steps of this notebook, but for now we will be using the same one.

In [ ]:
eig_rd = EPRanalysis(design, "hfss")

For the resonator analysis we will use the default setup. Youn can feel free to edit it the same way we did in section 1.

#### Execute simulation and verify convergence and EM field
Analyze the readout in isolation. Select the readout and terminate it with an open on both ends. Note that we are selecting for this analysis both the `readout` component and the `open_to_ground` component. The `open_to_ground` compoent might feel redundant because we are specifying in that open in the `open_terminations`, and the end converging reult is indeed the same. however the `open_to_ground` appears to help the system to ceonverge faster, so we keep it in there.

In [ ]:
eig_rd.sim.run(name="Readout",
               components=['readout', 'open_to_ground'],
               open_terminations=[('readout', 'start'), ('readout', 'end')])
eig_rd.sim.plot_convergences()

In [ ]:
eig_rd.sim.save_screenshot()  # optional

Recover eigenmode frequencies for each variation.

In [ ]:
eig_rd.get_frequencies()

Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:
eig_rd.sim.plot_fields('main')
eig_rd.sim.save_screenshot()

#### Refine

If convergence is not complete, or the EM field is unclear, update the number of passes and re-run the flow (below repeated for convenience)

In [ ]:
eig_rd.sim.setup.max_passes = 15   # update single setting
eig_rd.sim.run()
eig_rd.sim.plot_convergences()

Display the Ansys modeler window again and plot the E-field on the chip's surface with this updated number of passes. <br>
Note that the bright areas have become much smoother compared to the previous image, indicating better convergence.

In [ ]:
eig_rd.sim.plot_fields('main')
eig_rd.sim.save_screenshot()

### EPR Analysis
Find the electric and magnetic energy stored in the readout system.

In [ ]:
eig_rd.run_epr(no_junctions = True)

# 3. Analyze the combined transmon + CPW resonator system

### Finite Element Eigenmode Analysis

#### Setup

Create a separate analysis object for the combined qbit+readout.

In [ ]:
eig_qres = EPRanalysis(design, "hfss")

For the resonator analysis we look for 2 eigenmodes - one with stronger fields near the transmon, the other with stronger fields near the resonator. Therefore let's update the setup accordingly.

In [ ]:
eig_qres.sim.setup.n_modes = 2
eig_qres.sim.setup

#### Execute simulation and verify convergence and EM field
Analyze the qubit+readout. Select the qubit and the readout, then finalize with open termination on the other pins.

In [ ]:
eig_qres.sim.run(name="TransmonResonator",
                 components=['Q1', 'readout', 'open_to_ground'],
                 open_terminations=[('readout', 'end')])
eig_qres.sim.plot_convergences()

In [ ]:
eig_qres.sim.save_screenshot()  # optional

Display the Ansys modeler window again and plot the E-field on the chip's surface. you can select which of the two modes to visualize.

In [ ]:
eig_qres.sim.plot_fields('main', eigenmode=1)
eig_qres.sim.save_screenshot()

### EPR Analysis

Similarly to section 1, we need to pass to the renderer the names of the shapes that identify the junction (`rect` and `line`). These should be the same as in section 1, or you can look again for those in the renderer.

In [ ]:
eig_qres.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qres.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'
eig_qres.setup

We will now run epr as a single step. On screen you will observe various information in this order:
* stored energy = Electric and magnetic energy stored in the substrate and the system as a whole.
* EPR analysis results for all modes/variations.
* Spectrum analysis.
* Hamiltonian report.

In [ ]:
eig_qres.run_epr()

Once you are sure you are done with the qubit analysis, please explicitly release the Ansys session to allow for a smooth close of the external tool.

In [ ]:
eig_qb.sim.close()

In [ ]:
eig_rd.sim.close()

In [ ]:
eig_qres.sim.close()

# 4. Analyze a coupled 2-transmon system
### Create the design

This is a different system than the one analyzed in sections 1,2,3. Therefore, let's start by deleting the design currntly in the Qiskit Metal GUI (if any).

In [ ]:
design.delete_all_components()

Next, we create the `TwoTransmon` design, consisting of 2 transmons connected by a short coupler.

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x = '1.0 mm',
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=-1,loc_H=+1, pad_width='200um')
    )))

coupler = RouteStraight(design, 'coupler', Dict(hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='Q2', pin='readout')), ))

gui.rebuild()
gui.autoscale()

Let's observe the current table describing the junctions in the qiskit metal design

In [ ]:
design.qgeometry.tables['junction']

You can observe in the table above that every junction has been assigned a default inductance, capacitance and resistance values, based on the originating component class `default_options`. In this example we intend to replace those values with a variable name, which will later be set directly in the renderer. Therefore, let's proceed with updating these values in the qubit instances, and then propagate the update to the table with a `rebuild()`.
After executing the cell below, you can observe the change by re-executing the cell above.

In [ ]:
# TODO: fold this inside either an analysis class method, or inside the analysis class setup

qcomps = design.components  # short handle (alias)
qcomps['Q1'].options['hfss_inductance'] = 'Lj1'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj1'
qcomps['Q2'].options['hfss_inductance'] = 'Lj2'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj2'
gui.rebuild()  # line needed to propagate the updates from the qubit instance into the junction design table
gui.autoscale()

### Finite Element Eigenmode Analysis

#### Setup

Let's start the analysis by creating the appropriate analysis object.

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_2qb = EPRanalysis(design, "hfss")

Now let us update the setup of this analysis to reflect what we plan to do:
* define the variables that we have assigned to the inductance and capacitance of the junctions;
* increase accuracy of the convergence;
* observe the eigenmode corresponding to both qubits.

In [ ]:
eig_2qb.sim.setup.max_passes = 15
eig_2qb.sim.setup.max_delta_f = 0.05
eig_2qb.sim.setup.n_modes = 2
eig_2qb.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF',
                           Lj2= '9 nH', Cj2= '0 fF')
eig_2qb.sim.setup

By default, the analysis will be done on all components that we will list in the `run_sim()` method, but the analysis needs to know how much of the ground plane around the qubit to consider. One could use the declared chip dimension by passing the parameter `bux_plus_buffer = False` to the `run_sim()` method. However, its default (when said parameter is omitted) is to consider the ground plane to be as big as the minimum enclosing rectangle plus a set buffer. The default buffer value is `200um`, while in the cell below we will increase as an example that buffer to `500um`.

In [ ]:
# TODO: fold this inside either an analysis class method, or inside the analysis class setup

eig_2qb.sim.renderer.options['x_buffer_width_mm'] = 0.5
eig_2qb.sim.renderer.options['y_buffer_width_mm'] = 0.5
eig_2qb.sim.renderer.options

Let's finally run the cap extraction simulation and observe the convergence.

In [ ]:
eig_2qb.sim.run(name="TwoTransmons",
                components=['coupler', 'Q1', 'Q2'])

In [ ]:
eig_2qb.sim.plot_convergences()

In [ ]:
eig_2qb.sim.save_screenshot()  # optional

Display the Ansys modeler window again and plot the E-field on the chip's surface. Since we have analyzed 2 modes, you will need to select which mode to visualize. The default is mode 1, but the mode can inclusively be any integer between 1 and `setup.n_modes`.

In [ ]:
eig_2qb.sim.plot_fields('main', eigenmode=2)
eig_2qb.sim.save_screenshot()

### EPR Analysis
#### Setup
Identify the non-linear (Josephson) junctions in the model. in this case there are 2 junctions, which we will refer to as `jj1` and `jj2`. Also define the dissipative reference shapes. Remove the default junction and create the two.

In [ ]:
del eig_2qb.setup.junctions['jj']

In [ ]:
eig_2qb.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                  Lj_variable='Lj1', Cj_variable='Cj1')
eig_2qb.setup.junctions.jj2 = Dict(rect='JJ_rect_Lj_Q2_rect_jj', line='JJ_Lj_Q2_rect_jj_',
                  Lj_variable='Lj2', Cj_variable='Cj2')
eig_2qb.setup.sweep_variable = 'Lj1'
eig_2qb.setup

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
eig_2qb.run_epr()

Release Ansys's session

In [ ]:
eig_2qb.sim.close()

(optional) **final wrap**: Close the gui by removing the # in the line below.

In [ ]:
# gui.main_window.close()